# Beam Pairing Notebook

In [ ]:
import h5py
import simplekml
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from osgeo import ogr
import fiona

# Packages for kml2df (temporary addition for testing)
import os
import numpy as np
import pandas as pd
import re
import datetime

In [ ]:
# Read collection of RGT tracks
rgt_paths = '/home/jovyan/shared-public/ICESat-2-Hackweek/ground_tracks/cycle21_1hz_points.csv'
rgt_tracks = pd.read_csv(rgt_paths)
rgt_tracks

In [ ]:
# Approximate location of left and right beams
rgt_tracks['right_lon'] = rgt_tracks['lon'] + 3300./(np.cos(np.radians(rgt_tracks['lat']))*111000)
rgt_tracks['left_lon'] = rgt_tracks['lon'] - 3300./(np.cos(np.radians(rgt_tracks['lat']))*111000)
rgt_tracks.head()

In [ ]:
# Generate Point geometries for each of the beam pairs
from shapely import Point, MultiPoint
rgt_tracks['geom1'] = [Point(x, y) for x, y in zip(rgt_tracks.lon, rgt_tracks.lat)]
rgt_tracks['geom2'] = [Point(x, y) for x, y in zip(rgt_tracks.right_lon, rgt_tracks.lat)]
rgt_tracks['geom3'] = [Point(x, y) for x, y in zip(rgt_tracks.left_lon, rgt_tracks.lat)]
rgt_tracks.head()

In [ ]:
# Look at only a single RGT
single_rgt_track = rgt_tracks[(rgt_tracks['rgt']==1356) | (rgt_tracks['rgt']==472) | (rgt_tracks['rgt']==1097)]
single_rgt_track.head()

In [ ]:
len(single_rgt_track)

In [ ]:
# Combine Point geometries into Multipoint object
single_rgt_track['multi'] = [MultiPoint([x, y, z]) for x, y, z in zip(single_rgt_track.geom1, 
                                                                 single_rgt_track.geom2,
                                                                 single_rgt_track.geom3)]
single_rgt_track.head()

In [ ]:
len(single_track_gdf)

In [ ]:
# Set the MultiPoint geometry, and remove redundant columns
single_track_gdf = single_rgt_track.set_geometry('multi')
single_track_gdf = single_track_gdf.drop(['geometry','geom1','geom2', 'geom3'], axis=1)

In [ ]:
# Save geodataframe to a geoJSON
single_track_gdf.to_file('rgt472_rgt1097_rgt1356_allbeams.geojson', driver='GeoJSON')

In [ ]:
# Read the geoJSON back in
data = gpd.read_file('rgt472_rgt1097_rgt1356_allbeams.geojson')

In [ ]:
# View CRS of track information
data.crs

In [ ]:
# View ICESat-2 tracks over ESRI map
data.explore(tiles='Esri.WorldImagery')

In [ ]:
data.geometry

In [ ]:
test_data = pd.read_csv('/home/jovyan/icesat2-snowex/atl03_snowdepth_rgt1097_acp_202234.csv', header=0)
test_data.head()

In [ ]:
test_data_gpd = gpd.GeoDataFrame(test_data, geometry=gpd.points_from_xy(test_data.lon, test_data.lat),
                                 crs='EPSG:4326')

In [ ]:
%matplotlib widget
f, ax = plt.subplots()
data.plot(ax=ax)
test_data_gpd.plot(ax=ax)

In [ ]:
search_results_file = 'search_circle_ll_results.geojson'
search_result_gpd = gpd.read_file(search_results_file)
search_result_gpd.head()

In [ ]:
search_result_tracks = search_result_gpd[search_result_gpd['rgt']!='none']

In [ ]:
search_result_tracks['coords'] = search_result_tracks.geometry.apply(lambda geom: list(geom.coords))

In [ ]:
search_result_tracks['points'] = search_result_tracks.apply(lambda x: [y for y in x['geometry'].coords], axis=1)

In [ ]:
search_result_points = search_result_tracks[search_result_tracks.geometry.geom_type=='Point']

In [ ]:
search_result_points['lon'] = search_result_points.geometry.apply(lambda p: p.x)
search_result_points['lat'] = search_result_points.geometry.apply(lambda p: p.y)

In [ ]:
search_result_points['right_lon'] = search_result_points['lon'] + 3300./(np.cos(np.radians(search_result_points['lat']))*111000)
search_result_points['left_lon'] = search_result_points['lon'] - 3300./(np.cos(np.radians(search_result_points['lat']))*111000)
search_result_points.head()

In [ ]:
search_result_points['geom1'] = [Point(x, y) for x, y in zip(search_result_points.lon, search_result_points.lat)]
search_result_points['geom2'] = [Point(x, y) for x, y in zip(search_result_points.right_lon, search_result_points.lat)]
search_result_points['geom3'] = [Point(x, y) for x, y in zip(search_result_points.left_lon, search_result_points.lat)]
search_result_points.head()

In [ ]:
search_result_points['multi'] = [MultiPoint([x, y, z]) for x, y, z in zip(search_result_points.geom1, 
                                                                 search_result_points.geom2,
                                                                 search_result_points.geom3)]
search_result_points.head()

In [ ]:
search_result_gdf = search_result_points.set_geometry('multi')
search_result_gdf = search_result_gdf.drop(['geometry','geom1','geom2', 'geom3', 'coords'], axis=1)

In [ ]:
search_result_gdf = search_result_gdf.drop(['points'], axis=1)

In [ ]:
# Save geodataframe to a geoJSON
search_result_gdf.to_file('search_result_allbeams.geojson', driver='GeoJSON')

In [ ]:
# Read the geoJSON back in
data = gpd.read_file('search_result_allbeams.geojson')

In [ ]:
# Check out the search results in a map (currently only shows points)
data.explore()

Note that the above file reads the data in as Point geometries. I did not have enough time to convert them back to the LineString format.

Since we want 3 beams preferably in the same Geometry object, a MultiLineString would be preferable.